In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
inpatient_specialty = pd.read_csv('/home/bhatti/dataset/VCHAMPS/inpatient_specialty_train.csv').iloc[:,1:]

In [3]:
inpatient_specialty['Specialty start date'] = inpatient_specialty['Specialty start date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['Specialty end date'] = inpatient_specialty['Specialty end date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['period'] = preprocess_data.get_period(inpatient_specialty['Specialty start date'],inpatient_specialty['Specialty end date'])

In [4]:
specialty_class = preprocess_data.generate_dict_from_csv('/home/vivi/FDA_datasets/specialty_class.csv','Specialty','Class')

In [5]:
inpatient_specialty['Specialty encoded'] = preprocess_data.preprocess_substitute_categories(specialty_class, 
                                                                               inpatient_specialty['Specialty'])

### Merge with inpatient data
Filter out specialty time before last discharge date

In [6]:
path = '/home/hassan/lily/MLA/FDA/outpatient_mortality.csv'
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient.head()

,Internalpatientid,num_visits,CVD,last_visit_date,Age 00-20,Age 20-40,Age 40-60,Age 60-80,Age 80-100,Age 100-120,age_mean,age_std,freq,Died,away_from_death,died_within_125days
0,1,1483,1,2024-07-02 21:13:18,0,0,26,1440,17,0,71.708835,5.016889,64.48,0,NaN,0
1,2,2149,1,2024-04-05 03:30:10,0,0,269,1880,0,0,65.058152,4.662247,93.43,0,NaN,0
2,3,156,0,2005-10-11 09:10:49,0,0,0,87,69,0,79.892190,1.238569,31.20,1,0.0,1
3,4,336,1,2015-02-24 04:43:51,0,0,0,47,289,0,81.903713,3.210646,22.40,1,0.0,1
4,5,124,0,2004-11-04 14:48:32,0,0,0,124,0,0,75.693176,1.203346,17.71,1,0.0,1


In [7]:
inpatient_date = inpatient[['Internalpatientid','last_visit_date']]
inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
inpatient_date

,Internalpatientid,last_visit_date
0,1,2024-07-02 21:13:18
1,2,2024-04-05 03:30:10
2,3,2005-10-11 09:10:49
3,4,2015-02-24 04:43:51
4,5,2004-11-04 14:48:32
...,...,...
133232,169060,2017-09-26 11:10:08
133233,169061,2024-01-29 12:14:06
133234,169062,2005-10-04 20:08:10
133235,169063,2006-02-04 23:24:49


In [8]:
inpatient_specialty2 = inpatient_specialty.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
inpatient_specialty3 = inpatient_specialty2.loc[inpatient_specialty2['Specialty end date'] <= inpatient_specialty2['last_visit_date']]
inpatient_specialty3

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State,period,Specialty encoded,last_visit_date
0,1,78.725684,2022-12-31 05:41:51,2023-01-01 23:06:29,DERMATOLOGY,Indiana,1 days 17:24:38,0.0,2024-07-02 21:13:18
1,100001,86.568969,2012-07-21 23:00:00,2012-07-22 19:18:47,PM&R TRANSITIONAL REHAB,North Carolina,0 days 20:18:47,1.0,2014-12-19 09:23:59
2,100015,56.646451,2002-07-16 06:40:13,2002-07-16 06:40:13,INTERMEDIATE MEDICINE,Massachusetts,0 days 00:00:00,0.0,2005-08-03 09:14:52
3,100015,58.247495,2004-02-21 11:01:35,2004-02-22 14:56:40,DOMICILIARY PTSD,Rhode Island,1 days 03:55:05,1.0,2005-08-03 09:14:52
4,100019,81.174959,1999-10-24 10:44:24,1999-10-31 10:20:31,PLASTIC SURGERY,Florida,6 days 23:36:07,0.0,1999-11-01 02:51:07
...,...,...,...,...,...,...,...,...,...
1452904,9999,67.689758,2020-10-27 14:33:32,2020-10-27 14:33:32,MEDICAL OBSERVATION,Utah,0 days 00:00:00,0.0,2022-11-20 11:19:29
1452905,99993,58.013747,2007-02-18 00:35:16,2007-02-18 04:55:16,METABOLIC,Kentucky,0 days 04:20:00,0.0,2013-08-31 00:21:42
1452906,99994,83.090126,2016-10-18 10:35:16,2016-10-21 11:02:57,NH SHORT STAY DEMENTIA CARE,Texas,3 days 00:27:41,1.0,2020-04-30 16:27:49
1452907,99994,85.951737,2019-08-30 08:38:03,2019-09-05 09:28:48,"PULMONARY, NON-TB",Texas,6 days 00:50:45,NaN,2020-04-30 16:27:49


In [9]:
inpatient_specialty3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1411833 entries, 0 to 1452908
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype          
---  ------                --------------    -----          
 0   Internalpatientid     1411833 non-null  int64          
 1   Age at specialty      1411833 non-null  float64        
 2   Specialty start date  1411833 non-null  datetime64[ns] 
 3   Specialty end date    1411833 non-null  datetime64[ns] 
 4   Specialty             1411833 non-null  object         
 5   State                 1411833 non-null  object         
 6   period                1411833 non-null  timedelta64[ns]
 7   Specialty encoded     1358098 non-null  float64        
 8   last_visit_date       1411833 non-null  object         
dtypes: datetime64[ns](2), float64(2), int64(1), object(3), timedelta64[ns](1)
memory usage: 107.7+ MB


In [10]:
inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('count')

specialty_medical_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 0).sum()).reset_index(name='specialty medical count')

specialty_support_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 1).sum()).reset_index(name='specialty support count')

inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('mean')

/tmp/ipykernel_254759/3919990082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\
/tmp/ipykernel_254759/3919990082.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\


In [11]:
period_mean = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))

period_std = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_std['period'] = period_std['period'].apply(lambda x: np.nanstd(x.total_seconds()/86400,ddof=0))

period_mean = period_mean.rename(columns={'period': 'period mean'})
period_std = period_std.rename(columns={'period': 'period std'})

/tmp/ipykernel_254759/1693156475.py:2: RuntimeWarning: Mean of empty slice
  period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [12]:
inpatient_specialty4 = inpatient_specialty3[['Internalpatientid', 'mean age at specailty','specialty count']].sort_values('Internalpatientid')
inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)
inpatient_specialty4 = inpatient_specialty4.drop_duplicates().reset_index(drop = True)
inpatient_specialty4

/tmp/ipykernel_254759/554132651.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)


,Internalpatientid,mean age at specailty,specialty count
0,1,71.760299,8
1,2,65.896816,55
2,3,78.748857,3
3,4,84.177706,3
4,5,76.452471,4
...,...,...,...
83415,169055,58.918427,4
83416,169057,82.335103,77
83417,169060,65.595637,27
83418,169062,72.665887,24


In [13]:
inpatient_specialty5 = inpatient_specialty4.merge(period_mean, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(period_std, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.merge(specialty_medical_count, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(specialty_support_count, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.dropna(subset = 'period mean')
inpatient_specialty5

,Internalpatientid,mean age at specailty,specialty count,period mean,period std,specialty medical count,specialty support count
0,1,71.760299,8,1.985007,2.286435,8,0
1,2,65.896816,55,1.695425,3.832509,51,3
2,3,78.748857,3,81.500222,162.085913,1,2
3,4,84.177706,3,2.383800,1.709373,3,0
4,5,76.452471,4,1.407809,1.351702,2,2
...,...,...,...,...,...,...,...
83415,169055,58.918427,4,1.617095,1.735104,1,3
83416,169057,82.335103,77,1.277754,2.100649,58,19
83417,169060,65.595637,27,1.681511,3.024752,20,7
83418,169062,72.665887,24,6.870090,11.462599,18,6


In [14]:
inpatient_specialty5.to_csv('/home/vivi/FDA_datasets/inpatient_specialty_allcause_mortality.csv')